In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
## Imports and environment variables 
import torch, wandb, os, pandas as pd 
from travis_attack.utils import set_seed, set_session_options, setup_logging, setup_parser, update_config_with_parsed_arguments, resume_wandb_run, display_all, print_important_cfg_vars
from travis_attack.config import Config
from travis_attack.models import prepare_models, get_optimizer
from travis_attack.data import ProcessedDataset
from travis_attack.trainer import Trainer
from travis_attack.insights import (postprocess_df, create_and_log_wandb_postrun_plots, get_training_dfs)
from fastcore.basics import in_jupyter

import logging 
logger = logging.getLogger("run")

import warnings
warnings.filterwarnings("ignore", message="Passing `max_length` to BeamSearchScorer is deprecated")  # we ignore the warning because it works anyway for diverse beam search 

In [ ]:
from nbdev.export import notebook2script
notebook2script()

!jupyter nbconvert \
    --TagRemovePreprocessor.enabled=True \
    --TagRemovePreprocessor.remove_cell_tags="['hide']" \
    --TemplateExporter.exclude_markdown=True \
    --to python "run.ipynb"

Converted 00_utils.ipynb.
Converted 02_tests.ipynb.
Converted 03_config.ipynb.
Converted 07_models.ipynb.
Converted 10_data.ipynb.
Converted 20_trainer.ipynb.
Converted 25_insights.ipynb.
Converted Untitled.ipynb.
Converted baselines.ipynb.
Converted baselines_analysis.ipynb.
Converted index.ipynb.
Converted pp_eval_baselines.ipynb.
Converted run.ipynb.
Converted show_examples.ipynb.
Converted test_grammar_options.ipynb.
Converted test_pp_model.ipynb.
[NbConvertApp] Converting notebook run.ipynb to python


In [ ]:
cfg = Config()  # default values
if not in_jupyter():  # override with any -- options when running with command line
    parser = setup_parser()
    newargs = vars(parser.parse_args())
    cfg = update_config_with_parsed_arguments(cfg, newargs)
if cfg.use_small_ds:  cfg = cfg.small_ds()
set_seed(cfg.seed)
set_session_options()
setup_logging(cfg, disable_other_loggers=True)
vm_tokenizer,vm_model,pp_tokenizer,pp_model,ref_pp_model,sts_model,nli_tokenizer,nli_model,cola_tokenizer,cola_model,cfg = prepare_models(cfg)
optimizer = get_optimizer(cfg, pp_model)
ds = ProcessedDataset(cfg, vm_tokenizer, vm_model, pp_tokenizer, sts_model, load_processed_from_file=False)

travis_attack.data: INFO     Will load dataset financial with use_small_ds set to True
travis_attack.data: INFO     Will load dataset financial with use_small_ds set to True
travis_attack.data: INFO     Will load dataset financial with use_small_ds set to True
travis_attack.data: INFO     Will load dataset financial with use_small_ds set to True
travis_attack.data: INFO     Will load dataset financial with use_small_ds set to True


In [ ]:
cfg.wandb['mode'] = 'online'
trainer = Trainer(cfg, vm_tokenizer,vm_model,pp_tokenizer,pp_model,ref_pp_model,sts_model,nli_tokenizer,nli_model,cola_tokenizer,cola_model, optimizer,
         ds)
print_important_cfg_vars(cfg)
trainer.train()

{'pp_name': 'prithivida/parrot_paraphraser_on_T5',
 'dataset_name': 'financial',
 'sts_name': 'sentence-transformers/paraphrase-MiniLM-L12-v2',
 'nli_name': 'howey/electra-small-mnli',
 'cola_name': 'textattack/albert-base-v2-CoLA',
 'vm_name': 'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis',
 'seed': 421,
 'use_small_ds': True,
 'lr': 4e-05,
 'reward_fn': 'reward_fn_contradiction_and_letter_diff',
 'reward_clip_max': 4,
 'reward_vm_multiplier': 12,
 'sts_threshold': 0.8,
 'acceptability_threshold': 0.5,
 'contradiction_threshold': 0.2,
 'pp_letter_diff_threshold': 30,
 'reward_penalty_type': 'kl_div',
 'kl_coef': 0.25,
 'ref_logp_coef': None,
 'min_pp_length': 2,
 'max_pp_length': 48,
 'n_eval_seq': 48,
 'decode_method_train': 'sample',
 'decode_method_eval': 'sample',
 'gen_params_train': {'do_sample': True, 'temperature': 1.1, 'top_p': 0.95},
 'gen_params_eval': {'num_return_sequences': 48,
                     'do_sample': True,
                     'num_beams':

acceptability_scores-mean-test,▁
acceptability_scores-mean-train,▁
acceptability_scores-mean-valid,▁
acceptability_scores-std-test,▁
acceptability_scores-std-train,▁
acceptability_scores-std-valid,▁
any_adv_example_proportion-test,▁
any_adv_example_proportion-train,▁
any_adv_example_proportion-valid,▁
contradiction_scores-mean-test,▁
contradiction_scores-mean-train,▁


wandb: wandb version 0.12.17 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


travis_attack.trainer: INFO     Launching initial eval run: train
travis_attack.trainer: INFO     Launching initial eval run: train
travis_attack.trainer: INFO     Launching initial eval run: train
travis_attack.trainer: INFO     Launching initial eval run: train
travis_attack.trainer: INFO     Launching initial eval run: train


travis_attack.trainer: INFO     Launching initial eval run: valid
travis_attack.trainer: INFO     Launching initial eval run: valid
travis_attack.trainer: INFO     Launching initial eval run: valid
travis_attack.trainer: INFO     Launching initial eval run: valid
travis_attack.trainer: INFO     Launching initial eval run: valid


...

travis_attack.trainer: INFO     Now on epoch 9 of 10
travis_attack.trainer: INFO     Now on epoch 9 of 10
travis_attack.trainer: INFO     Now on epoch 9 of 10
travis_attack.trainer: INFO     Now on epoch 9 of 10
travis_attack.trainer: INFO     Now on epoch 9 of 10


travis_attack.trainer: INFO     Epoch: 9. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 9. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 9. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 9. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 9. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Eval metric: 0.250 | Running median: 0.250
travis_attack.trainer: INFO     Eval metric: 0.250 | Running median: 0.250
travis_attack.trainer: INFO     Eval metric: 0.250 | Running median: 0.250
travis_attack.trainer: INFO     Eval metric: 0.250 | Running median: 0.250
travis_attack.trainer: INFO     Eval metric: 0.250 | Running median: 0.250


travis_attack.trainer: INFO     Now on epoch 10 of 10
travis_attack.trainer: INFO     Now on epoch 10 of 10
travis_attack.trainer: INFO     Now on epoch 10 of 10
travis_attack.trainer: INFO     Now on epoch 10 of 10
travis_attack.trainer: INFO     Now on epoch 10 of 10


travis_attack.trainer: INFO     Epoch: 10. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 10. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 10. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 10. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 10. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Eval metric: 0.250 | Running median: 0.250
travis_attack.trainer: INFO     Eval metric: 0.250 | Running median: 0.250
travis_attack.trainer: INFO     Eval metric: 0.250 | Running median: 0.250
travis_attack.trainer: INFO     Eval metric: 0.250 | Running median: 0.250
travis_attack.trainer: INFO     Eval metric: 0.250 | Running median: 0.250


travis_attack.trainer: INFO     Evaluating test set with best model at path : ../model_checkpoints/travis_attack/summer-cosmos-178/model_0.pt
travis_attack.trainer: INFO     Evaluating test set with best model at path : ../model_checkpoints/travis_attack/summer-cosmos-178/model_0.pt
travis_attack.trainer: INFO     Evaluating test set with best model at path : ../model_checkpoints/travis_attack/summer-cosmos-178/model_0.pt
travis_attack.trainer: INFO     Evaluating test set with best model at path : ../model_checkpoints/travis_attack/summer-cosmos-178/model_0.pt
travis_attack.trainer: INFO     Evaluating test set with best model at path : ../model_checkpoints/travis_attack/summer-cosmos-178/model_0.pt


In [ ]:
%debug

> /data/tproth/travis_attack/travis_attack/trainer.py(180)_training_step()
    178                 return_dict_in_generate=True, output_scores=True), # remove_invalid_values=False,
    179                # pad_token_id = self.pp_tokenizer.pad_token_id, eos_token_id = self.pp_tokenizer.eos_token_id
--> 180             pp_l = self.pp_tokenizer.batch_decode(pp_output.sequences, skip_special_tokens=True)
    181         # Update_batch_size_and_length_variables
    182         self.orig_batch_size,self.orig_length =   data['input_ids'].shape[0],   data['input_ids'].shape[1]

ipdb> pp_output
(SampleEncoderDecoderOutput(sequences=tensor([[    0,   486,     8,   337,    97,    27,   183,  5010,    24,    62,
           130,     3,   179,    12,  8582,     8,  1470,    28,    69,   293,
           372,     5,     1,     0,     0,     0,     0,     0],
        [    0,    37,   349,    31,     7,  2337,    16,  5769,    54,    36,
           894,    16,     8,  4019,  1364,  1596,    13,     8,  



ipdb> ll
    166     def _training_step(self, data, raw):
    167         """Forward pass, loss function, backwards pass, parameter update (with gradient accumulation optional),
    168         recording results, wandb logging.
    169         """
    170         if not self.pp_model.training: self.pp_model.train()
    171         if not self.vm_model.training: self.vm_model.train()
    172         for k in ['input_ids', 'attention_mask', 'label', "orig_truelabel_probs", "orig_sts_embeddings"]: data[k] = data[k].to(self._cfg.device)
    173         with timecode() as self.batch_time_d['time_generate_pp']:
    174             pp_output = self.pp_model.generate_with_grad(
    175                 input_ids=data['input_ids'], attention_mask=data['attention_mask'],
    176                 num_return_sequences=1, num_beams=1, **self._cfg.gen_params_train,
    177                 min_length=self._cfg.min_pp_length, max_length=self._cfg.max_pp_length,
    178                 return_dict_in_g

In [ ]:
#df = pd.read_csv(cfg.path_results + "run_results.csv")
#display_all(df)

In [ ]:
# df = pd.read_csv(f'{cfg.path_run}training_step.csv')
# #display_all(df.query('idx==1'))
# df.columns

In [ ]:
#trainer.run.finish()

In [ ]:
# # ## TO RESUME RUN
# cfg = Config()
# cfg.run_id = 'b9r88lmz'
# cfg.run_name = "fresh-cosmos-151"
# cfg.path_run = f"{cfg.path_checkpoints}{cfg.run_name}/"
# run = resume_wandb_run(cfg)


In [ ]:
cfg.path_run

In [ ]:
df_d = get_training_dfs(cfg.path_run, postprocessed=False)
for k, df in df_d.items(): 
    df_d[k] = postprocess_df(df, filter_idx=None, num_proc=1)
    df_d[k].to_pickle(f"{cfg.path_run}{k}_postprocessed.pkl")    
create_and_log_wandb_postrun_plots(df_d)
trainer.run.finish()
#run.finish()